In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
K.set_image_dim_ordering('th')
import pickle

Using TensorFlow backend.


In [2]:
num_classes = bird_audio_with_label = pickle.load( open( "bed.txt", "rb" ) )
X_bird = bird_audio_with_label["bed"]
Y_bird = np.array([[1,0]] * X_bird.shape[0])
print X_bird.shape
print Y_bird.shape

bed_audio_with_label = pickle.load( open( "bird.txt", "rb" ) )
X_bed = bed_audio_with_label["bird"]
Y_bed = np.array([[0,1]] * X_bed.shape[0])
print X_bed.shape
print Y_bed.shape

X = np.vstack((X_bird, X_bed))
Y = np.vstack((Y_bird, Y_bed))
print X.shape
print Y.shape

(1713, 20, 100)
(1713, 2)
(1731, 20, 100)
(1731, 2)
(3444, 20, 100)
(3444, 2)


In [3]:
X = X.reshape(X.shape[0], 1, 20, 100)
print X.shape
print Y.shape
num_classes = Y.shape[1]
print num_classes

(3444, 1, 20, 100)
(3444, 2)
2


In [4]:
model = Sequential()
model.add(Conv2D(30, (5, 5), input_shape=(1, 20, 100), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 16, 96)        780       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 8, 48)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 6, 46)         4065      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 3, 23)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 3, 23)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1035)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               132608    
__________

In [6]:
file_path="CNN_val_loss_bird_dog.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)
callbacks_list = [checkpoint, early] #early
model.fit(X, Y, validation_split=0.1, callbacks=callbacks_list, epochs=20, batch_size= 100)

Train on 3099 samples, validate on 345 samples
Epoch 1/20
3099/3099 [==============================] - 21s - loss: 1.0040 - acc: 0.6947 - val_loss: 0.4406 - val_acc: 0.7739
Epoch 2/20
3099/3099 [==============================] - 20s - loss: 0.2656 - acc: 0.8948 - val_loss: 0.2688 - val_acc: 0.8870
Epoch 3/20
3099/3099 [==============================] - 20s - loss: 0.1754 - acc: 0.9355 - val_loss: 0.2623 - val_acc: 0.9101
Epoch 4/20
3099/3099 [==============================] - 20s - loss: 0.1407 - acc: 0.9474 - val_loss: 0.2204 - val_acc: 0.9246
Epoch 5/20
3099/3099 [==============================] - 21s - loss: 0.1222 - acc: 0.9568 - val_loss: 0.1416 - val_acc: 0.9507
Epoch 6/20
3099/3099 [==============================] - 20s - loss: 0.1090 - acc: 0.9616 - val_loss: 0.2364 - val_acc: 0.9217
Epoch 7/20
3099/3099 [==============================] - 22s - loss: 0.0853 - acc: 0.9690 - val_loss: 0.2168 - val_acc: 0.9333
Epoch 8/20
3099/3099 [==============================] - 20s - loss: 0.0

In [7]:
model = load_model(file_path)

In [8]:
predicted = model.predict(X)

In [9]:
from sklearn import metrics

In [10]:
metrics.log_loss(Y, predicted)

0.083990980919505673